<a href="https://www.kaggle.com/code/aggarwalbhavya/failed-orders-at-gett?scriptVersionId=191830138" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
# importing the necessary libraries
import pandas as pd
import numpy as np

# Problem Statements

Please complete the following tasks.

**1**: Build up distribution of orders according to reasons for failure: cancellations before and after driver assignment, and reasons for order rejection. Analyse the resulting plot. Which category has the highest number of orders?

**2**: Plot the distribution of failed orders by hours. Is there a trend that certain hours have an abnormally high proportion of one category or another? What hours are the biggest fails? How can this be explained?

**3**: Plot the average time to cancellation with and without driver, by the hour. If there are any outliers in the data, it would be better to remove them. Can we draw any conclusions from this plot?

**4**: Plot the distribution of average ETA by hours. How can this plot be explained?

# Data Order CSV Analysis

**order_datetime**: time of the order

**origin_longitude** and **origin_latitude**: longitude and latitude of order request

**m_order_eta**: time before order arrival (ETA)

**order_gk**: order number

**order_status_key**: status enum having a mapping as follows-
* 4: cancelled by client
* 9: cancelled by system aka reject

**is_driver_assigned_key**: indicator wheteher a driver has been assigned or not
**cancellation_time_in_seconds**: how many seconds passed before cancellation

In [4]:
# reading the data orders 
data_orders = pd.read_csv('/kaggle/input/gett-insights/data_orders.csv')
data_orders.head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,cancellations_time_in_seconds
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,4,1,198.0
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0
2,12:07:50,-0.969520,51.455544,477.0,3000582891479,4,1,46.0
3,13:50:20,-1.054671,51.460544,658.0,3000582941169,4,1,62.0
4,21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN


In [5]:
data_orders.shape

(10716, 8)

# Data Offer CSV Analysis

**order_gk**: order number associated with the "orders" dataset

**offer_id**: ID of an offer

In [6]:
# reading the data offers
data_offers = pd.read_csv('/kaggle/input/gett-insights/data_offers.csv')
data_offers.head()

,order_gk,offer_id
0,3000579625629,300050936206
1,3000627306450,300052064651
2,3000632920686,300052408812
3,3000632771725,300052393030
4,3000583467642,300051001196


In [7]:
data_offers.shape

(334363, 2)

# Data Cleaning and Preparation

In [8]:
data_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_datetime                 10716 non-null  object 
 1   origin_longitude               10716 non-null  float64
 2   origin_latitude                10716 non-null  float64
 3   m_order_eta                    2814 non-null   float64
 4   order_gk                       10716 non-null  int64  
 5   order_status_key               10716 non-null  int64  
 6   is_driver_assigned_key         10716 non-null  int64  
 7   cancellations_time_in_seconds  7307 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 669.9+ KB


In [9]:
# showcases that what data point has null value 
data_orders.isnull().sum()

order_datetime                      0
origin_longitude                    0
origin_latitude                     0
m_order_eta                      7902
order_gk                            0
order_status_key                    0
is_driver_assigned_key              0
cancellations_time_in_seconds    3409
dtype: int64

As per the dataset description, we deduce that data points ***order_status_key*** and ***is_driver_assigned_key*** aren't traditional forms of explanatory data points and rather indicate to an internal data marking sysytem which could lead to communication of jargon. So, we can replace these data entry points with their literal *String* meanings.

Our desired output can be achieved in several methods as listed below:

**Method 1**:

is_driver_assigned = np.where(data_orders['is_driver_assigned_key'] == 1, "Yes", "No")

order_status = np.where(data_orders['order_status_key'] == 4, "Client Cancelled", "System Rejected")

**Method 2**: Using Lambda fucntion in apply

is_driver_assigned = data_orders['is_driver_assigned_key'].apply(lambda x: "Yes" if x == 1 else "No")

order_status = data_orders['order_status_key'].apply(lambda x: "Client Cancelled" if x == 4 else "System Rejected")

**Method 3**: Using maps and dictionaries

driver_map = {1: "Yes", 0: "No"}

status_map = {4: "Client Cancelled", 9: "System Rejected"}

is_driver_assigned = data_orders['is_driver_assigned_key'].map(driver_map)

order_status = data_orders['order_status_key'].map(status_map)

**Method 4**: Using replace

is_driver_assigned = data_orders['is_driver_assigned_key'].replace(driver_map)

order_status = data_orders['order_status_key'].replace(status_map)

**Update the data_orders with adding these new columns and removing the old ones**

In [10]:
# added data columns by cleaning data
data_orders['is_driver_assigned'] = np.where(data_orders['is_driver_assigned_key'] == 1, "Yes", "No")
data_orders['order_status'] = np.where(data_orders['order_status_key'] == 4, "Client Cancelled", "System Rejected")

# removing redundant data
data_orders.drop(columns = ['is_driver_assigned_key', 'order_status_key'], inplace = True)

In [11]:
# checking for updated changes
data_orders.head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,cancellations_time_in_seconds,is_driver_assigned,order_status
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,198.0,Yes,Client Cancelled
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,128.0,No,Client Cancelled
2,12:07:50,-0.969520,51.455544,477.0,3000582891479,46.0,Yes,Client Cancelled
3,13:50:20,-1.054671,51.460544,658.0,3000582941169,62.0,Yes,Client Cancelled
4,21:24:45,-0.967605,51.458236,NaN,3000583140877,NaN,No,System Rejected


Another problem statement requires us to distribute the failure of orders by hour so we can clean **order_datetime** to extract the **hour** component for our analysis.

In [12]:
data_orders['order_datetime'].dtype

dtype('O')

In [13]:
# Since the datetime is stored as stype Object aka String, to extract, we can fetch the 
# hour component simply by splitting the string
data_orders['order_hour'] = data_orders['order_datetime'].str.split(":").apply(lambda time: time[0])

In [14]:
data_orders.head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,cancellations_time_in_seconds,is_driver_assigned,order_status,order_hour
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,198.0,Yes,Client Cancelled,18
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,128.0,No,Client Cancelled,20
2,12:07:50,-0.969520,51.455544,477.0,3000582891479,46.0,Yes,Client Cancelled,12
3,13:50:20,-1.054671,51.460544,658.0,3000582941169,62.0,Yes,Client Cancelled,13
4,21:24:45,-0.967605,51.458236,NaN,3000583140877,NaN,No,System Rejected,21


# Creating a new dataset with data points required for analysis

Creating a new dataset without non-useable data points and filling missing data values. We drop **m_order_eta** here because of its high missing data value count.

In [15]:
data_orders_cleaned = data_orders.drop(columns = ['order_datetime', 'origin_longitude', 'origin_latitude', 'm_order_eta'])

In [16]:
data_orders_cleaned.head()

,order_gk,cancellations_time_in_seconds,is_driver_assigned,order_status,order_hour
0,3000583041974,198.0,Yes,Client Cancelled,18
1,3000583116437,128.0,No,Client Cancelled,20
2,3000582891479,46.0,Yes,Client Cancelled,12
3,3000582941169,62.0,Yes,Client Cancelled,13
4,3000583140877,NaN,No,System Rejected,21


In [17]:
data_orders_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_gk                       10716 non-null  int64  
 1   cancellations_time_in_seconds  7307 non-null   float64
 2   is_driver_assigned             10716 non-null  object 
 3   order_status                   10716 non-null  object 
 4   order_hour                     10716 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 418.7+ KB


Since, our analysis requires average plot of **cancellations_time_in_seconds**, we will be filling the missing data values with average based on **order_hour** and **is_driver_assigned**.

In [18]:
data_orders_cleaned['cancellations_time_in_seconds'] = data_orders_cleaned['cancellations_time_in_seconds'].fillna(data_orders_cleaned.groupby(['order_hour', 'is_driver_assigned'])['cancellations_time_in_seconds'].transform('mean'))
data_orders_cleaned.head()

,order_gk,cancellations_time_in_seconds,is_driver_assigned,order_status,order_hour
0,3000583041974,198.000000,Yes,Client Cancelled,18
1,3000583116437,128.000000,No,Client Cancelled,20
2,3000582891479,46.000000,Yes,Client Cancelled,12
3,3000582941169,62.000000,Yes,Client Cancelled,13
4,3000583140877,113.959821,No,System Rejected,21


In [19]:
data_orders_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_gk                       10716 non-null  int64  
 1   cancellations_time_in_seconds  10716 non-null  float64
 2   is_driver_assigned             10716 non-null  object 
 3   order_status                   10716 non-null  object 
 4   order_hour                     10716 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 418.7+ KB


# Creating separate ETA dataset

New Dataframe for ETA column

In [27]:
eta_column = { 'order_eta': data_orders['m_order_eta'], 'order_hour': data_orders['order_hour'] }
eta_dataframe = pd.DataFrame(data = eta_column)
eta_dataframe.head()

,order_eta,order_hour
0,60.0,18
1,NaN,20
2,477.0,12
3,658.0,13
4,NaN,21


In [28]:
eta_dataframe = eta_dataframe.dropna()

In [29]:
eta_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2814 entries, 0 to 10715
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_eta   2814 non-null   float64
 1   order_hour  2814 non-null   object 
dtypes: float64(1), object(1)
memory usage: 66.0+ KB


# Downloading Cleaned Up Datasets for Visualizations

In [ ]:
eta_dataframe.to_excel('/content/drive/MyDrive/Colab Notebooks/Failed Order Gett/eta_only.xlsx')

In [ ]:
data_orders_cleaned.to_excel('/content/drive/MyDrive/Colab Notebooks/Failed Order Gett/data_orders.xlsx')